In [1]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d sandhyakrishnan02/china-covid-cases-and-deaths

100%|██████████| 7.62k/7.62k [00:00<00:00, 8.41MB/s]


In [2]:
!unzip china-covid-cases-and-deaths.zip

Archive:  china-covid-cases-and-deaths.zip
  inflating: China_Covid.csv         


# China Covid Cases using RNN

## Getting data

In [7]:
import pandas as pd
import numpy as np 

In [8]:
df = pd.read_csv("China_Covid.csv")
df.head()

,Date,Total Cases,Total Death
0,05 Jan 23,4613307,17336
1,04 Jan 23,4596884,17273
2,03 Jan 23,4580698,17198
3,02 Jan 23,4560468,17124
4,01 Jan 23,4537107,17062


## Cleaning Data

In [9]:
df.drop("Date", axis=1, inplace=True) 


In [10]:
df.isna().sum()

Total Cases    0
Total Death    0
dtype: int64

## Preprocessing Data

In [50]:
training_set = df.iloc[:, 0:1].values
training_set

array([[   17],
       [   18],
       [   26],
       ...,
       [17198],
       [17273],
       [17336]])

In [52]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)
training_set_scaled

array([[0.00000000e+00],
       [5.77400543e-05],
       [5.19660488e-04],
       ...,
       [9.92031873e-01],
       [9.96362377e-01],
       [1.00000000e+00]])

In [53]:
X = []
y = []

for i in range(60, len(df)):
  X.append(training_set_scaled[i-60:i, 0])
  y.append(training_set_scaled[i, 0])

X ,y = np.array(X), np.array(y)

In [54]:
X

array([[0.00000000e+00, 5.77400543e-05, 5.19660488e-04, ...,
        1.86615855e-01, 1.86846816e-01, 1.87193256e-01],
       [5.77400543e-05, 5.19660488e-04, 1.44350136e-03, ...,
        1.86846816e-01, 1.87193256e-01, 1.88059357e-01],
       [5.19660488e-04, 1.44350136e-03, 2.25186212e-03, ...,
        1.87193256e-01, 1.88059357e-01, 1.88059357e-01],
       ...,
       [9.03227669e-01, 9.03862810e-01, 9.04902131e-01, ...,
        9.81176742e-01, 9.84179225e-01, 9.87759108e-01],
       [9.03862810e-01, 9.04902131e-01, 9.05652751e-01, ...,
        9.84179225e-01, 9.87759108e-01, 9.92031873e-01],
       [9.04902131e-01, 9.05652751e-01, 9.05941452e-01, ...,
        9.87759108e-01, 9.92031873e-01, 9.96362377e-01]])

In [55]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
X

array([[[0.00000000e+00],
        [5.77400543e-05],
        [5.19660488e-04],
        ...,
        [1.86615855e-01],
        [1.86846816e-01],
        [1.87193256e-01]],

       [[5.77400543e-05],
        [5.19660488e-04],
        [1.44350136e-03],
        ...,
        [1.86846816e-01],
        [1.87193256e-01],
        [1.88059357e-01]],

       [[5.19660488e-04],
        [1.44350136e-03],
        [2.25186212e-03],
        ...,
        [1.87193256e-01],
        [1.88059357e-01],
        [1.88059357e-01]],

       ...,

       [[9.03227669e-01],
        [9.03862810e-01],
        [9.04902131e-01],
        ...,
        [9.81176742e-01],
        [9.84179225e-01],
        [9.87759108e-01]],

       [[9.03862810e-01],
        [9.04902131e-01],
        [9.05652751e-01],
        ...,
        [9.84179225e-01],
        [9.87759108e-01],
        [9.92031873e-01]],

       [[9.04902131e-01],
        [9.05652751e-01],
        [9.05941452e-01],
        ...,
        [9.87759108e-01],
        [9.9203

## Traning Model

In [56]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [61]:
regressor = Sequential()

In [62]:
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [63]:
regressor.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
32/32 [==============================] - 9s 99ms/step - loss: 0.1189
Epoch 2/100
32/32 [==============================] - 3s 97ms/step - loss: 0.0816
Epoch 3/100
32/32 [==============================] - 3s 106ms/step - loss: 0.0779
Epoch 4/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0785
Epoch 5/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0770
Epoch 6/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0758
Epoch 7/100
32/32 [==============================] - 3s 98ms/step - loss: 0.0752
Epoch 8/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0750
Epoch 9/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0747
Epoch 10/100
32/32 [==============================] - 3s 99ms/step - loss: 0.0747
Epoch 11/100
32/32 [==============================] - 3s 100ms/step - loss: 0.0745
Epoch 12/100
32/32 [==============================] - 3s 98ms/step - loss: 0.0742
Epoch 13/100
32/32 [===